## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,66.11,61,81,5.99,broken clouds
1,1,Koath,IN,25.3167,84.2667,91.31,5,1,5.48,clear sky
2,2,Oriximina,BR,-1.7656,-55.8661,78.17,90,89,3.29,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,81.50,75,100,22.82,overcast clouds
4,4,Pangnirtung,CA,66.1451,-65.7125,-0.40,92,100,5.75,mist


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Oriximina,BR,-1.7656,-55.8661,78.17,90,89,3.29,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,81.50,75,100,22.82,overcast clouds
7,7,Rikitea,PF,-23.1203,-134.9692,77.65,70,16,16.40,few clouds
13,13,Vaini,TO,-21.2000,-175.2000,80.85,74,100,11.14,overcast clouds
16,16,Pocone,BR,-16.2567,-56.6228,83.75,64,0,3.76,clear sky
18,18,Caravelas,BR,-17.7125,-39.2481,85.78,60,39,4.63,scattered clouds
21,21,Townsville,AU,-19.2500,146.8000,79.68,74,75,8.05,broken clouds
22,22,Mitsamiouli,KM,-11.3847,43.2844,83.64,70,100,2.01,overcast clouds
25,25,Ugoofaaru,MV,5.6667,73.0000,84.20,68,11,7.20,few clouds
32,32,Atuona,PF,-9.8000,-139.0333,77.88,78,29,19.04,light rain


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Oriximina,BR,78.17,overcast clouds,-1.7656,-55.8661,
3,Butaritari,KI,81.50,overcast clouds,3.0707,172.7902,
7,Rikitea,PF,77.65,few clouds,-23.1203,-134.9692,
13,Vaini,TO,80.85,overcast clouds,-21.2000,-175.2000,
16,Pocone,BR,83.75,clear sky,-16.2567,-56.6228,
18,Caravelas,BR,85.78,scattered clouds,-17.7125,-39.2481,
21,Townsville,AU,79.68,broken clouds,-19.2500,146.8000,
22,Mitsamiouli,KM,83.64,overcast clouds,-11.3847,43.2844,
25,Ugoofaaru,MV,84.20,few clouds,5.6667,73.0000,
32,Atuona,PF,77.88,light rain,-9.8000,-139.0333,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.count()

City                   219
Country                219
Max Temp               219
Current Description    219
Lat                    219
Lng                    219
Hotel Name             219
dtype: int64

In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10).dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Oriximina,BR,78.17,overcast clouds,-1.7656,-55.8661,Pousada Tia Amélia
3,Butaritari,KI,81.50,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
7,Rikitea,PF,77.65,few clouds,-23.1203,-134.9692,People ThankYou
13,Vaini,TO,80.85,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
16,Pocone,BR,83.75,clear sky,-16.2567,-56.6228,Hotel e Churrascaria Pantaneira
18,Caravelas,BR,85.78,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
21,Townsville,AU,79.68,broken clouds,-19.2500,146.8000,Rydges Southbank Townsville
22,Mitsamiouli,KM,83.64,overcast clouds,-11.3847,43.2844,Foyer ADM
25,Ugoofaaru,MV,84.20,few clouds,5.6667,73.0000,Ungoofaaru Inn
32,Atuona,PF,77.88,light rain,-9.8000,-139.0333,Villa Enata


In [28]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))